# １章 ウェブの注文数を分析する１０本ノック

ここでは、ある企業のECサイトでの商品の注文数の推移を分析していきます。  
データの属性を理解し、分析をするためにデータを加工した後、  
データの可視化を行うことで問題を発見していくプロセスを学びます。

### ノック１：データを読み込んでみよう

In [32]:
import pandas as pd
customer_master = pd.read_csv('customer_master.csv')
customer_master.head()


,customer_id,customer_name,registration_date,customer_name_kana,email,gender,age,birth,pref
0,IK152942,平田 裕次郎,2019-01-01 00:25:33,ひらた ゆうじろう,hirata_yuujirou@example.com,M,29,1990/6/10,石川県
1,TS808488,田村 詩織,2019-01-01 01:13:45,たむら しおり,tamura_shiori@example.com,F,33,1986/5/20,東京都
2,AS834628,久野 由樹,2019-01-01 02:00:14,ひさの ゆき,hisano_yuki@example.com,F,63,1956/1/2,茨城県
3,AS345469,鶴岡 薫,2019-01-01 04:48:22,つるおか かおる,tsuruoka_kaoru@example.com,M,74,1945/3/25,東京都
4,GD892565,大内 高史,2019-01-01 04:54:51,おおうち たかし,oouchi_takashi@example.com,M,54,1965/8/5,千葉県


pandas.read_csv
https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

pandas.DataFrame.head
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html

In [33]:
item_master = pd.read_csv('item_master.csv')
item_master.head()

,item_id,item_name,item_price
0,S001,PC-A,50000
1,S002,PC-B,85000
2,S003,PC-C,120000
3,S004,PC-D,180000
4,S005,PC-E,210000


In [34]:
transaction_1 = pd.read_csv('transaction_1.csv')
transaction_1.head()

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865


In [35]:
transaction_detail_1 = pd.read_csv('transaction_detail_1.csv')
transaction_detail_1.head()

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1
3,3,T0000000116,S005,1
4,4,T0000000117,S002,2


・データの先頭5行を表示させることで、どのようなデータ列が存在するのか、それぞれのデータ列の関係性など、データの大枠を掴むことができる。

・データの概要を捉えて、分析に適した形に加工することから始める。

### ノック２：データを結合(ユニオン)してみよう

In [36]:
transaction_2 = pd.read_csv('transaction_2.csv')
#transaction_2.head()
transaction = pd.concat([transaction_1, transaction_2], ignore_index = True)
transaction.head()

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865


pandas.concat
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

In [37]:
print(len(transaction_1))
print(len(transaction_2))
print(len(transaction_1)+len(transaction_2))
print(len(transaction))

5000
1786
6786
6786


In [38]:
transaction_detail_2 = pd.read_csv('transaction_detail_2.csv')
#transaction_2.head()
transaction_detail = pd.concat([transaction_detail_1, transaction_detail_2], ignore_index = True)
transaction_detail.head()

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1
3,3,T0000000116,S005,1
4,4,T0000000117,S002,2


In [39]:
print(len(transaction_detail_1))
print(len(transaction_detail_2))
print(len(transaction_detail_1)+len(transaction_detail_2))
print(len(transaction_detail))

5000
2144
7144
7144


### ノック３：売上データ同士を結合(ジョイン)してみよう

In [40]:
join_data = pd.merge(transaction_detail, transaction[['transaction_id', 'payment_date', 'customer_id']], on = 'transaction_id', how = 'left')
join_data.head()

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502
1,1,T0000000114,S001,1,2019-02-01 01:37:23,HD678019
2,2,T0000000115,S003,1,2019-02-01 02:34:19,HD298120
3,3,T0000000116,S005,1,2019-02-01 02:47:23,IK452215
4,4,T0000000117,S002,2,2019-02-01 04:33:46,PL542865


pandas.merge
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html

In [41]:
transaction[['transaction_id', 'payment_date', 'customer_id']]


,transaction_id,payment_date,customer_id
0,T0000000113,2019-02-01 01:36:57,PL563502
1,T0000000114,2019-02-01 01:37:23,HD678019
2,T0000000115,2019-02-01 02:34:19,HD298120
3,T0000000116,2019-02-01 02:47:23,IK452215
4,T0000000117,2019-02-01 04:33:46,PL542865
...,...,...,...
6781,T0000006894,2019-07-31 21:20:44,HI400734
6782,T0000006895,2019-07-31 21:52:48,AS339451
6783,T0000006896,2019-07-31 23:35:25,OA027325
6784,T0000006897,2019-07-31 23:39:35,TS624738


In [42]:
transaction.loc[:, ['transaction_id', 'payment_date', 'customer_id']]


,transaction_id,payment_date,customer_id
0,T0000000113,2019-02-01 01:36:57,PL563502
1,T0000000114,2019-02-01 01:37:23,HD678019
2,T0000000115,2019-02-01 02:34:19,HD298120
3,T0000000116,2019-02-01 02:47:23,IK452215
4,T0000000117,2019-02-01 04:33:46,PL542865
...,...,...,...
6781,T0000006894,2019-07-31 21:20:44,HI400734
6782,T0000006895,2019-07-31 21:52:48,AS339451
6783,T0000006896,2019-07-31 23:35:25,OA027325
6784,T0000006897,2019-07-31 23:39:35,TS624738


In [43]:
print(len(transaction_detail))
print(len(transaction))
print(len(join_data))

7144
6786
7144


### ノック４：マスタデータを結合(ジョイン)してみよう

In [44]:
join_data = pd.merge(join_data, customer_master, on = 'customer_id', how = 'left')
join_data.head()

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name,registration_date,customer_name_kana,email,gender,age,birth,pref
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,井本 芳正,2019-01-07 14:34:35,いもと よしまさ,imoto_yoshimasa@example.com,M,30,1989/7/15,熊本県
1,1,T0000000114,S001,1,2019-02-01 01:37:23,HD678019,三船 六郎,2019-01-27 18:00:11,みふね ろくろう,mifune_rokurou@example.com,M,73,1945/11/29,京都府
2,2,T0000000115,S003,1,2019-02-01 02:34:19,HD298120,山根 小雁,2019-01-11 08:16:02,やまね こがん,yamane_kogan@example.com,M,42,1977/5/17,茨城県
3,3,T0000000116,S005,1,2019-02-01 02:47:23,IK452215,池田 菜摘,2019-01-10 05:07:38,いけだ なつみ,ikeda_natsumi@example.com,F,47,1972/3/17,兵庫県
4,4,T0000000117,S002,2,2019-02-01 04:33:46,PL542865,栗田 憲一,2019-01-25 06:46:05,くりた けんいち,kurita_kenichi@example.com,M,74,1944/12/17,長崎県


In [45]:
join_data = pd.merge(join_data, item_master, on = 'item_id', how = 'left')
join_data.head()

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name,registration_date,customer_name_kana,email,gender,age,birth,pref,item_name,item_price
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,井本 芳正,2019-01-07 14:34:35,いもと よしまさ,imoto_yoshimasa@example.com,M,30,1989/7/15,熊本県,PC-E,210000
1,1,T0000000114,S001,1,2019-02-01 01:37:23,HD678019,三船 六郎,2019-01-27 18:00:11,みふね ろくろう,mifune_rokurou@example.com,M,73,1945/11/29,京都府,PC-A,50000
2,2,T0000000115,S003,1,2019-02-01 02:34:19,HD298120,山根 小雁,2019-01-11 08:16:02,やまね こがん,yamane_kogan@example.com,M,42,1977/5/17,茨城県,PC-C,120000
3,3,T0000000116,S005,1,2019-02-01 02:47:23,IK452215,池田 菜摘,2019-01-10 05:07:38,いけだ なつみ,ikeda_natsumi@example.com,F,47,1972/3/17,兵庫県,PC-E,210000
4,4,T0000000117,S002,2,2019-02-01 04:33:46,PL542865,栗田 憲一,2019-01-25 06:46:05,くりた けんいち,kurita_kenichi@example.com,M,74,1944/12/17,長崎県,PC-B,85000


ただし、売上(price)が結合できていない状態

### ノック5：必要なデータ列を作ろう

In [46]:
join_data['price']  = join_data['quantity'] * join_data['item_price']
join_data[['quantity', 'item_price', 'price']].head()


,quantity,item_price,price
0,1,210000,210000
1,1,50000,50000
2,1,120000,120000
3,1,210000,210000
4,2,85000,170000


データを結合したりする度に、件数の確認などを行うこと

### ノック6：データ検算をしよう

In [48]:
print(join_data["price"].sum())
print(transaction["price"].sum())

971135000
971135000


In [ ]:
print(join_data["price"].sum())
print(transaction["price"].sum())

In [49]:
print(join_data["price"].sum()) == print(transaction["price"].sum())

971135000
971135000


True

### ノック7：各種統計量を把握しよう

In [54]:
join_data.isnull().sum()


detail_id             0
transaction_id        0
item_id               0
quantity              0
payment_date          0
customer_id           0
customer_name         0
registration_date     0
customer_name_kana    0
email                 0
gender                0
age                   0
birth                 0
pref                  0
item_name             0
item_price            0
price                 0
dtype: int64

In [52]:
join_data.describe()


,detail_id,quantity,age,item_price,price
count,7144.000000,7144.000000,7144.000000,7144.000000,7144.000000
mean,3571.500000,1.199888,50.265677,121698.628219,135937.150056
std,2062.439494,0.513647,17.190314,64571.311830,68511.453297
min,0.000000,1.000000,20.000000,50000.000000,50000.000000
25%,1785.750000,1.000000,36.000000,50000.000000,85000.000000
50%,3571.500000,1.000000,50.000000,102500.000000,120000.000000
75%,5357.250000,1.000000,65.000000,187500.000000,210000.000000
max,7143.000000,4.000000,80.000000,210000.000000,420000.000000


In [56]:
print(join_data["payment_date"].min())
print(join_data["payment_date"].max())

2019-02-01 01:36:57
2019-07-31 23:41:38


### ノック8：月別でデータを集計してみよう

In [59]:
join_data.dtypes

detail_id              int64
transaction_id        object
item_id               object
quantity               int64
payment_date          object
customer_id           object
customer_name         object
registration_date     object
customer_name_kana    object
email                 object
gender                object
age                    int64
birth                 object
pref                  object
item_name             object
item_price             int64
price                  int64
dtype: object

In [62]:
join_data['payment_date'] = pd.to_datetime(join_data['payment_date'])

join_data["payment_month"] = join_data["payment_date"].dt.strftime("%Y%m")
join_data[["payment_date", "payment_month"]].head()


,payment_date,payment_month
0,2019-02-01 01:36:57,201902
1,2019-02-01 01:37:23,201902
2,2019-02-01 02:34:19,201902
3,2019-02-01 02:47:23,201902
4,2019-02-01 04:33:46,201902


### ノック9：月別、商品別でデータを集計してみよう

### ノック10：商品別の売上推移を可視化してみよう